In [1]:
%load_ext watermark
%watermark -a Chan -d -v -p numpy,pandas,konlpy,tensorflow

Chan 2019-08-12 

CPython 3.7.3
IPython 7.6.1

numpy 1.16.4
pandas 0.24.2
konlpy 0.5.1
tensorflow 1.13.1


In [2]:
import numpy as np
import pandas as pd
from konlpy.tag import Okt
import tensorflow as tf
from tensorflow.python.keras.preprocessing import sequence
from tensorflow import keras
from tqdm import tqdm
from collections import Counter
import pickle

In [3]:
# 각 동영상별로 count해야한다. 일단 dump파일로 짜놓기

In [4]:
datasets = pd.DataFrame()
for i in range(9,10):
    print(i)
    dump = pd.read_csv("../../data/labeled ({}).csv".format(i), engine='python')
    datasets = pd.concat([datasets, dump])
datasets.head()

9


,comment,url_id,label
0,궁,23,0
1,진에어 화이팅!,11,0
2,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,12,0
3,싹둑이 채팅창 관리 중입니다,34,0
4,앗,52,0


In [5]:
datasets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 3 columns):
comment    2998 non-null object
url_id     3000 non-null int64
label      3000 non-null int64
dtypes: int64(2), object(1)
memory usage: 70.4+ KB


In [6]:
datasets.fillna('null',inplace=True)

In [7]:
len(datasets)

3000

In [8]:
morp = Okt()
morped = [morp.morphs(_, norm=True, stem=True) for _ in tqdm(datasets.comment)]

C:\Users\skarn\Anaconda3\envs\DataAnalysis\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)
100%|█████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:26<00:00, 111.26it/s]


In [9]:
morped[:5]

[['궁'],
 ['진에어', '화이팅', '!'],
 ['ㅋㅋㅋ'],
 ['싹둑', '이', '채팅', '창', '관리', '중', '이다'],
 ['앗']]

In [10]:
# load vocab
with open('./vocab/vocab_index.pickle', 'rb') as f:
    vocab_index = pickle.load(f)
with open('./vocab/vocab_inverted_index.pickle', 'rb') as f:
    vocab_inverted_index = pickle.load(f)

In [11]:
def text_to_index(tokens):
    oov_id = 1
    indexes = []
    for tok in tokens:
        if tok in vocab_index:
            indexes.append(vocab_index[tok])
        else:
            indexes.append(oov_id)

    return indexes

In [12]:
pad_id = 0

x_variable = [text_to_index(_) for _ in morped]

sentence_size = 10
x_padded = sequence.pad_sequences(x_variable,
                                 maxlen=sentence_size,
                                 truncating='post',
                                 padding='post',
                                 value=pad_id)

# Load Model 

In [13]:
new_model = keras.models.load_model('../model/cnn_oversample.h5')
new_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 10)          100000    
_________________________________________________________________
conv1d (Conv1D)              (None, None, 32)          992       
_________________________________________________________________
dropout (Dropout)            (None, None, 32)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 32)          3104      
_________________________________________________________________
global_max_pooling1d (Globa

In [14]:
datasets['predict'] = new_model.predict_classes(x_padded)

In [15]:
new_model.metrics_names

['loss', 'recall']

In [16]:
new_model.evaluate(x_padded, pd.get_dummies(datasets.label)) # loss, acc

3000/3000 [==============================] - ETA: 43s - loss: 0.2186 - recall: 0.937 - ETA: 1s - loss: 0.4439 - recall: 0.900 - ETA: 0s - loss: 0.4356 - recall: 0.90 - ETA: 0s - loss: 0.4502 - recall: 0.89 - ETA: 0s - loss: 0.4183 - recall: 0.90 - 1s 236us/sample - loss: 0.4018 - recall: 0.9073


[0.4018002461095651, 0.9073333]

## 비속어 사전 기반 필터링의 경우

In [17]:
from auto_label import auto_label
import pandas as pd

In [18]:
result = auto_label(datasets,'./word-fobidden.json')

In [19]:
result.head()

,comment,url_id,label,predict,filtered
0,궁,23,0,0,0
1,진에어 화이팅!,11,0,0,0
2,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,12,0,0,0
3,싹둑이 채팅창 관리 중입니다,34,0,0,0
4,앗,52,0,0,0


In [20]:
recall_data = result[result.label==1]

In [21]:
len(recall_data)

169

In [22]:
sum(recall_data.predict), sum(recall_data.predict==0)

(84, 85)

In [23]:
sum(recall_data.filtered==1), sum(recall_data.filtered==0)

(30, 139)

In [24]:
sum(recall_data.predict)/len(recall_data) # 모델 방법의 recall

0.4970414201183432

In [25]:
sum(recall_data.filtered)/len(recall_data) # 비속어사전 방법의 recall

0.17751479289940827

In [28]:
sum(result.predict == result.label)/len(result) # 모델 방법의 recall

0.9073333333333333

#### 전체 채팅 중 유해채팅 비율

In [27]:
datasets.label.sum() / len(datasets)

0.05633333333333333

> 8%가 적당하다

In [28]:
recall_data.to_csv('./result/cnn_result.csv', encoding='euc-kr')